# Module 1 - Foundational Models

This introductory module serves as a comprehensive primer on building functional AI agents using LangChain, moving systematically from basic initialization to complex multi-modal interactions. You will begin by learning to configure chat models and refine their behavior through system prompts and engineering techniques, before progressing to the critical step of integrating external tools to expand the agent's utility. The lab also covers the implementation of short-term memory for fluid conversations and the use of audio and visual inputs to create a more versatile assistant. Ultimately, the course culminates in a practical project where you will synthesize your skills to develop a web-connected culinary assistant capable of generating recipes from specific ingredients.

## 00. Getting started

Load the required environment variables:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

## 01. Initialising and invoking a model

The foundation of any agent is the chat model, described as the conductor of the orchestra or the system's "thinking brain". The nteraction with models should be straightforward and interchangeable.
Use the `AzureChatOpenAI` function to select a model (e.g., gpt-4o).

In our lab, the model has been defined using the `AZURE_OPENAI_DEPLOYMENT` environment variable:

In [2]:
!echo $AZURE_OPENAI_DEPLOYMENT

gpt-4o


In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

Then, you interact with the model by invoking it with a question. The response is returned as an AI message object, where the primary answer is stored within the content key.

In [11]:
response = model.invoke("What's the capital of the Moon?")

response

AIMessage(content='The Moon does not have a capital, as it is not governed or inhabited in a way that would necessitate a capital city. The Moon is an astronomical body and, under international law (specifically the Outer Space Treaty of 1967), it cannot be claimed by any nation. Human activity on the Moon so far has been limited to exploration and research, but no permanent settlements or governance have been established.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 14, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CqihmLzm5KLPOoZxtFEVwLUrZU1AR', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hat

In [12]:
print(response.content)

The Moon does not have a capital, as it is not governed or inhabited in a way that would necessitate a capital city. The Moon is an astronomical body and, under international law (specifically the Outer Space Treaty of 1967), it cannot be claimed by any nation. Human activity on the Moon so far has been limited to exploration and research, but no permanent settlements or governance have been established.


In [13]:
from pprint import pprint

pprint(response.response_metadata)

{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
                            'protected_material_code': {'detected': False,
                                                        'filtered': False},
                            'protected_material_text': {'detected': False,
                                                        'filtered': False},
                            'self_harm': {'filtered': False,
                                          'severity': 'safe'},
                            'sexual': {'filtered': False, 'severity': 'safe'},
                            'violence': {'filtered': False,
                                         'severity': 'safe'}},
 'finish_reason': 'stop',
 'id': 'chatcmpl-CqihmLzm5KLPOoZxtFEVwLUrZU1AR',
 'logprobs': None,
 'model_name': 'gpt-4o-2024-11-20',
 'model_provider': 'openai',
 'prompt_filter_results': [{'content_filter_results': {'hate': {'filtered': False,
                                                     

## 02. Customising your Model

Beyond the raw text response, models provide metadata (like token usage) and can be fine-tuned via specific parameters to control their behavior.
• Temperature: Controls randomness; higher numbers increase creativity, while lower numbers make the model more deterministic.
• Max Tokens: Limits the length of the output.
• Timeout & Retries: Timeout sets the maximum wait time for a response, and max_retries defines how many times the system should attempt a failed request.

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=1.0
)

response = model.invoke("What's the capital of the Moon?")
print(response.content)

The Moon does not have a capital, as it is not a governed entity or inhabited in the way that a country or nation is. However, it has been a subject of exploration and study, and locations like the Apollo landing sites or proposed lunar bases could be of interest in discussions about the Moon.


## 03. Model Providers

A key strength of LangChain is that it is model agnostic, meaning you can switch between different providers (OpenAI, Claude, Gemini) with minimal code changes.
You can swap an entire agent's "brain" by changing the model name in the initialization function or using specific provider libraries for newer models. Please refer to the following documentation to get details on supported providers: https://docs.langchain.com/oss/python/integrations/chat

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment="gpt-5.1",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

response = model.invoke("What's the capital of the Moon?")
print(response.content)

## 04. Initialising and invoking an agent

To move from a simple chat model to an agent, LangChain uses an abstraction built on top of LangGraph. The agent is initialized by passing the model as the primary argument.

In [ ]:
import os
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_agent

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

agent = create_agent(model=model)

Then, when invoking the agent, you pass a dictionary containing a human message function `Human Communication` to specify the user's input.

In [16]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What's the capital of the Moon?")]}
)

Unlike a simple model call, the agent returns a dictionary of messages representing the full conversation history; the final answer is always the last message in that list.

In [17]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content="What's the capital of the Moon?", additional_kwargs={}, response_metadata={}, id='80d21333-a4a4-4ea6-b5a0-2045bb94c560'),
              AIMessage(content='The Moon does not have a capital, as it is not governed by any nation or entity. However, various locations on the Moon, such as the Sea of Tranquility (where Apollo 11 landed), have significant historical and scientific importance.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 14, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CqjJxJntKSwGNmhWI5CnrlSLULO5j', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate':

In the next command, response['messages'][-1] selects the final message in the agent’s message list (the latest turn), and .content accesses that message’s text; print(...) outputs that text to the notebook’s stdout.

In [18]:
print(response['messages'][-1].content)

The Moon does not have a capital, as it is not governed by any nation or entity. However, various locations on the Moon, such as the Sea of Tranquility (where Apollo 11 landed), have significant historical and scientific importance.


Because the system accepts a dictionary, you can pass a list of previous Human and AI messages to provide context or "gaslight" the model with prior (even if false) information.

In [19]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What's the capital of the Moon?"),
    AIMessage(content="The capital of the Moon is Luna City."),
    HumanMessage(content="Interesting, tell me more about Luna City")]}
)

pprint(response)

{'messages': [HumanMessage(content="What's the capital of the Moon?", additional_kwargs={}, response_metadata={}, id='71142c73-ed3b-497d-8800-0544ffcb345f'),
              AIMessage(content='The capital of the Moon is Luna City.', additional_kwargs={}, response_metadata={}, id='b56ef9d9-7471-4bf2-b68d-e000f68709a4'),
              HumanMessage(content='Interesting, tell me more about Luna City', additional_kwargs={}, response_metadata={}, id='73c2df6f-031f-4a82-92ab-ecdcc3758cbe'),
              AIMessage(content='Well, Luna City doesn\'t exist—yet! It\'s a popular concept in science fiction and creative imaginings about how humanity might one day colonize the Moon. Writers, futurists, and scientists have proposed the idea of establishing permanent settlements or even cities on the Moon to serve as bases for research, tourism, mining, or as a stepping stone for further space exploration, like missions to Mars.\n\n"Luna City" often symbolizes the first major lunar habitat in many sci-fi

## 05. Streaming Output

As agent systems grow complex, they must handle multiple messages and the "perceived latency" of slow response times. Agent response times can jump from milliseconds to seconds or minutes. To improve the user experience, use streaming tokens (printing text as it is generated) rather than waiting for the full answer

In [20]:
for token, metadata in agent.stream(
    {"messages": [HumanMessage(content="Tell me all about Luna City, the capital of the Moon")]},
    stream_mode="messages"
):

    # token is a message chunk with token content
    # metadata contains which node produced the token
    
    if token.content:  # Check if there's actual content
        print(token.content, end="", flush=True)  # Print token

As of my knowledge cutoff in October 2023, "Luna City" does not exist as an actual place, as there is no established or permanent colony on the Moon. However, "Luna City" is a frequent feature in science fiction and speculative discussions about humans establishing permanent settlements on the Moon. It's often imagined as the capital of such a lunar colony. Below is some information combining scientific possibilities and fictional depictions:

---

### **1. Hypothetical Characteristics of Luna City**
In fiction and speculative thinking, Luna City is often depicted as:

- **Location**: Typically located in a stable region on the Moon, such as near the poles (e.g., the South Pole) due to the presence of more consistent sunlight for solar power and potential access to water ice in shadowed craters.
- **Architecture**: The city could be built underground or utilize regolith (lunar soil) to shield inhabitants from harmful radiation, micrometeorites, and extreme temperature differences.
- **